
# SuperTrend Strategy Backtester  
**(with Toggleable Confirmations, `min_confirmations`, and new ADX & EMA(9/21) options)**

This notebook runs a SuperTrend-first long strategy with optional confirmations and a flexible `min_confirmations` gate.

**Confirmations you can toggle on/off:**
- RSI filter (not overbought, optionally rising)
- MACD (line ≥ signal or recent bull cross)
- EMA **5/20** crossover
- **NEW:** EMA **9/21** crossover
- Bollinger momentum (close above rising upper band)
- OBV rising
- **NEW:** **ADX/DI** strength filter *(ADX ≥ threshold and +DI ≥ −DI)*

`min_confirmations` controls how many of the **enabled** confirmations must pass:
- `None` → require **all enabled** (original behavior)
- `0` → SuperTrend-only
- `N >= 1` → require **at least N**

**Exits**: SuperTrend bearish flip (recommended single exit), SL/TP, and optional indicator exits that mirror toggles (MACD bear, EMA 5/20 bear, **NEW:** EMA 9/21 bear, **NEW:** DI bear).


In [46]:

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os, json, math, warnings, logging
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd

try:
    import yfinance as yf
    import matplotlib.pyplot as plt
except Exception:
    pass

warnings.filterwarnings("ignore", category=FutureWarning)

# =========================
# LOGGING
# =========================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("supertrend_strategy_v2")


In [47]:

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    # Data
    start_date: str = "2015-01-01"
    end_date: str   = "2025-01-01"
    static_symbols: Optional[List[str]] = None
    static_symbols_path: Optional[str] = None
    cache_dir: str = "cache"
    out_dir: str   = "outputs"
    plot: bool     = True

    # --- SuperTrend core (mandatory) ---
    st_atr_len: int = 10
    st_multiplier: float = 3.0

    # --- Optional confirmations (toggles) ---
    use_rsi_filter: bool = True
    rsi_len: int = 14
    rsi_ob_level: float = 70.0
    rsi_require_rising: bool = True

    use_macd_confirm: bool = True
    macd_fast: int = 12
    macd_slow: int = 26
    macd_signal: int = 9
    macd_cross_lookback: int = 3

    use_ema_confirm: bool = True          # EMA(5/20)
    ema_fast: int = 5
    ema_slow: int = 20

    # NEW: EMA(9/21) toggle
    use_ema921_confirm: bool = False
    ema_fast_2: int = 9
    ema_slow_2: int = 21

    use_bb_reinforce: bool = True
    bb_len: int = 20
    bb_std: float = 2.0
    bb_require_rising: bool = True

    use_obv_confirm: bool = True
    obv_lookback: int = 5

    # NEW: ADX/DI toggle
    use_adx_confirm: bool = False
    adx_len: int = 14
    adx_min: float = 20.0   # typical thresholds: 20/25/30

    # --- At-least-N confirmations logic ---
    # None => require ALL enabled confirmations (original behavior)
    # 0 => ST-only; N => require at least N of the enabled confirmations
    min_confirmations: Optional[int] = None

    # Fixed exits
    stop_loss_pct: float = 0.10
    target_pct: float    = 0.10

    # Portfolio
    apply_fees: bool    = True
    initial_capital: float = 500_000.0
    max_concurrent_positions: int = 5
    deploy_cash_frac: float = 0.25
    top_k_daily: int = 300
    volar_lookback: int = 252
    filter_52w_window: int = 252
    within_pct_of_52w_high: float = 0.50

    # Execution
    entry_on_next_open: bool = True
    exit_on_next_open: bool = True

    # Optional liquidity guards (OFF by default)
    enable_basic_liquidity: bool = False
    min_price_inr: float = 50.0
    min_avg_vol_20d: float = 50_000.0

CFG = Config()


In [48]:

# =========================
# FEES
# =========================
APPLY_FEES = True

def calc_fees(turnover_buy: float, turnover_sell: float) -> float:
    if not APPLY_FEES:
        return 0.0
    BROKER_PCT = 0.001
    BROKER_MIN = 5.0
    BROKER_CAP = 20.0
    STT_PCT = 0.001
    STAMP_BUY_PCT = 0.00015
    EXCH_PCT = 0.0000297
    SEBI_PCT = 0.000001
    IPFT_PCT = 0.000001
    GST_PCT = 0.18
    DP_SELL = 20.0 if turnover_sell >= 100 else 0.0

    def _broker(turnover):
        if turnover <= 0:
            return 0.0
        fee = turnover * BROKER_PCT
        return max(BROKER_MIN, min(fee, BROKER_CAP))

    br_buy  = _broker(turnover_buy)
    br_sell = _broker(turnover_sell)
    stt   = STT_PCT * (turnover_buy + turnover_sell)
    stamp = STAMP_BUY_PCT * turnover_buy
    exch  = EXCH_PCT * (turnover_buy + turnover_sell)
    sebi  = SEBI_PCT * (turnover_buy + turnover_sell)
    ipft  = IPFT_PCT * (turnover_buy + turnover_sell)
    dp    = DP_SELL
    gst_base = br_buy + br_sell + dp + exch + sebi + ipft
    gst   = GST_PCT * gst_base
    return float((br_buy + br_sell) + stt + stamp + exch + sebi + ipft + dp + gst)


In [49]:

# =========================
# Helpers & Data
# =========================
def ensure_dirs(*paths):
    for p in paths:
        os.makedirs(p, exist_ok=True)

def today_str():
    return pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y-%m-%d")

def load_static_symbols(static_symbols: Optional[List[str]], static_symbols_path: Optional[str]) -> List[str]:
    syms: List[str] = []
    if static_symbols and len(static_symbols) > 0:
        syms = list(static_symbols)
    elif static_symbols_path and os.path.exists(static_symbols_path):
        with open(static_symbols_path, "r") as f:
            syms = [line.strip() for line in f if line.strip()]
    else:
        raise ValueError(
            "Provide CFG.static_symbols=[...] ('.NS' suffixes) or set CFG.static_symbols_path "
            "to a file containing one symbol per line."
        )
    out = []
    for s in syms:
        s = s.strip().upper()
        if not s.endswith(".NS"):
            s = f"{s}.NS"
        out.append(s)
    seen = set(); uniq = []
    for s in out:
        if s not in seen:
            uniq.append(s); seen.add(s)
    return uniq

def fetch_prices(tickers: List[str], start: str, end: Optional[str], cache_dir: str) -> Dict[str, pd.DataFrame]:
    ensure_dirs(cache_dir)
    data = {}
    end = end or today_str()
    for ticker in tickers:
        cache_path = os.path.join(cache_dir, f"{ticker.replace('^', '_')}.parquet")
        if os.path.exists(cache_path):
            try:
                df = pd.read_parquet(cache_path)
                if len(df) and pd.to_datetime(df.index[-1]).strftime("%Y-%m-%d") >= end:
                    data[ticker] = df; continue
            except Exception:
                pass
        try:
            df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False, multi_level_index=False)
            if df is None or df.empty:
                continue
            df = df.rename(columns=str.title)[['Open','High','Low','Close','Volume']].dropna()
            df.index.name = "date"
            df.to_parquet(cache_path)
            data[ticker] = df
        except Exception:
            continue
    return data


In [50]:

def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False, min_periods=span).mean()

def rsi(series: pd.Series, length: int = 14) -> pd.Series:
    delta = series.diff()
    gain = (delta.where(delta > 0, 0.0)).rolling(length).mean()
    loss = (-delta.where(delta < 0, 0.0)).rolling(length).mean()
    rs = gain / loss.replace(0.0, np.nan)
    out = 100 - (100 / (1 + rs))
    return out.fillna(50.0)

def macd(series: pd.Series, fast=12, slow=26, signal=9):
    ema_fast = series.ewm(span=fast, adjust=False, min_periods=fast).mean()
    ema_slow = series.ewm(span=slow, adjust=False, min_periods=slow).mean()
    line = ema_fast - ema_slow
    sig  = line.ewm(span=signal, adjust=False, min_periods=signal).mean()
    hist = line - sig
    return line, sig, hist

def bollinger(series: pd.Series, length=20, std=2.0):
    mid = series.rolling(length).mean()
    dev = series.rolling(length).std(ddof=0)
    upper = mid + std*dev
    lower = mid - std*dev
    return mid, upper, lower

def obv(close: pd.Series, volume: pd.Series) -> pd.Series:
    direction = np.sign(close.diff().fillna(0.0))
    return (direction * volume).cumsum().fillna(0.0)

def atr(high: pd.Series, low: pd.Series, close: pd.Series, length: int=14) -> pd.Series:
    prev_close = close.shift(1)
    tr = pd.concat([
        (high - low).abs(),
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)
    return tr.rolling(length).mean()

def supertrend(df: pd.DataFrame, atr_len=10, multiplier=3.0):
    hl2 = (df["High"] + df["Low"]) / 2.0
    _atr = atr(df["High"], df["Low"], df["Close"], length=atr_len)
    upperband = hl2 + multiplier * _atr
    lowerband = hl2 - multiplier * _atr

    st = pd.Series(index=df.index, dtype=float)
    direction = pd.Series(index=df.index, dtype=int)  # +1 bullish, -1 bearish

    st.iloc[0] = upperband.iloc[0]
    direction.iloc[0] = 1

    for i in range(1, len(df)):
        if df["Close"].iloc[i] > st.iloc[i-1]:
            direction.iloc[i] = 1
        elif df["Close"].iloc[i] < st.iloc[i-1]:
            direction.iloc[i] = -1
        else:
            direction.iloc[i] = direction.iloc[i-1]

        if direction.iloc[i] == 1:
            st.iloc[i] = min(upperband.iloc[i], st.iloc[i-1])
        else:
            st.iloc[i] = max(lowerband.iloc[i], st.iloc[i-1])

        if direction.iloc[i] == 1 and df["Close"].iloc[i] < st.iloc[i]:
            direction.iloc[i] = -1
            st.iloc[i] = lowerband.iloc[i]
        elif direction.iloc[i] == -1 and df["Close"].iloc[i] > st.iloc[i]:
            direction.iloc[i] = 1
            st.iloc[i] = upperband.iloc[i]

    return pd.DataFrame({"st_value": st, "st_dir": direction})


In [51]:

def _true_range(high: pd.Series, low: pd.Series, prev_close: pd.Series) -> pd.Series:
    return pd.concat([
        (high - low).abs(),
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)

def adx_di(high: pd.Series, low: pd.Series, close: pd.Series, length: int = 14):
    prev_high = high.shift(1)
    prev_low  = low.shift(1)
    prev_close = close.shift(1)

    up_move   = high - prev_high
    down_move = prev_low - low
    plus_dm  = up_move.where((up_move > down_move) & (up_move > 0), 0.0)
    minus_dm = down_move.where((down_move > up_move) & (down_move > 0), 0.0)

    tr = _true_range(high, low, prev_close)

    alpha = 1.0 / length
    atr = tr.ewm(alpha=alpha, adjust=False, min_periods=length).mean()
    plus_di  = 100 * (plus_dm.ewm(alpha=alpha, adjust=False, min_periods=length).mean() / atr)
    minus_di = 100 * (minus_dm.ewm(alpha=alpha, adjust=False, min_periods=length).mean() / atr)

    dx = 100 * (plus_di - minus_di).abs() / (plus_di + minus_di).replace(0, np.nan)
    adx_series = dx.ewm(alpha=alpha, adjust=False, min_periods=length).mean()
    return adx_series, plus_di, minus_di


In [52]:

def compute_indicators(df: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    d = df.copy()
    st = supertrend(d, atr_len=cfg.st_atr_len, multiplier=cfg.st_multiplier)
    d = pd.concat([d, st], axis=1)

    d["rsi"] = rsi(d["Close"], cfg.rsi_len)
    d["rsi_prev"] = d["rsi"].shift(1)

    d["ema_f"] = ema(d["Close"], cfg.ema_fast)
    d["ema_s"] = ema(d["Close"], cfg.ema_slow)

    d["ema_f2"] = ema(d["Close"], cfg.ema_fast_2)
    d["ema_s2"] = ema(d["Close"], cfg.ema_slow_2)

    d["macd_line"], d["macd_signal"], d["macd_hist"] = macd(d["Close"], cfg.macd_fast, cfg.macd_slow, cfg.macd_signal)
    d["bb_mid"], d["bb_up"], d["bb_dn"] = bollinger(d["Close"], cfg.bb_len, cfg.bb_std)

    d["obv"] = obv(d["Close"], d["Volume"])
    d["obv_prev"] = d["obv"].shift(1)

    d["adx"], d["di_plus"], d["di_minus"] = adx_di(d["High"], d["Low"], d["Close"], cfg.adx_len)

    d["avg_vol_20"] = d["Volume"].rolling(20).mean()
    d["high_52w"] = d["Close"].rolling(cfg.filter_52w_window).max()
    return d.dropna()


In [53]:

def basic_liquidity_ok(row: pd.Series, cfg: Config) -> bool:
    if not cfg.enable_basic_liquidity:
        return True
    if row["Close"] < cfg.min_price_inr:
        return False
    if row["avg_vol_20"] < cfg.min_avg_vol_20d:
        return False
    return True


In [54]:

def simulate_ticker(ticker: str, df: pd.DataFrame, cfg: Config):
    d = compute_indicators(df, cfg).copy()
    cols = ["ticker","side","date","price","shares","reason","signal_reason","score",
            "rsi","macd_line","macd_signal","ema_f","ema_s","ema_f2","ema_s2",
            "st_value","st_dir","bb_mid","bb_up","obv","adx","di_plus","di_minus","close","high_52w"]
    if d.empty:
        return pd.DataFrame(columns=cols), pd.Series(dtype=float)

    st_flip_up = (d["st_dir"].shift(1) == -1) & (d["st_dir"] == 1)

    rsi_ok = pd.Series(True, index=d.index)
    if cfg.use_rsi_filter:
        rsi_ok = (d["rsi"] < cfg.rsi_ob_level)
        if cfg.rsi_require_rising:
            rsi_ok = rsi_ok & (d["rsi"] >= d["rsi_prev"])

    macd_ok = pd.Series(True, index=d.index)
    if cfg.use_macd_confirm:
        macd_cross_up = (d["macd_line"].shift(1) <= d["macd_signal"].shift(1)) & (d["macd_line"] > d["macd_signal"])
        macd_recent_bull = macd_cross_up.rolling(cfg.macd_cross_lookback).max().fillna(False).astype(bool)
        macd_ok = (d["macd_line"] >= d["macd_signal"]) | macd_recent_bull

    ema_ok = pd.Series(True, index=d.index)          # 5/20
    if cfg.use_ema_confirm:
        ema_ok = d["ema_f"] > d["ema_s"]

    ema921_ok = pd.Series(True, index=d.index)       # 9/21
    if cfg.use_ema921_confirm:
        ema921_ok = d["ema_f2"] > d["ema_s2"]

    bb_ok = pd.Series(True, index=d.index)
    if cfg.use_bb_reinforce:
        bb_mid_rising = d["bb_mid"] >= d["bb_mid"].shift(1) if cfg.bb_require_rising else pd.Series(True, index=d.index)
        bb_ok = (d["Close"] >= d["bb_up"]) & bb_mid_rising

    obv_ok = pd.Series(True, index=d.index)
    if cfg.use_obv_confirm:
        obv_ok = d["obv"] >= d["obv_prev"].fillna(d["obv"])

    adx_ok = pd.Series(True, index=d.index)
    if cfg.use_adx_confirm:
        adx_ok = (d["adx"] >= cfg.adx_min) & (d["di_plus"] >= d["di_minus"])

    confirms = []
    if cfg.use_rsi_filter:        confirms.append(rsi_ok)
    if cfg.use_macd_confirm:      confirms.append(macd_ok)
    if cfg.use_ema_confirm:       confirms.append(ema_ok)
    if cfg.use_ema921_confirm:    confirms.append(ema921_ok)
    if cfg.use_bb_reinforce:      confirms.append(bb_ok)
    if cfg.use_obv_confirm:       confirms.append(obv_ok)
    if cfg.use_adx_confirm:       confirms.append(adx_ok)

    if len(confirms) == 0:
        confirmations_ok = pd.Series(True, index=d.index)
    else:
        if cfg.min_confirmations is None:
            conf = confirms[0]
            for c in confirms[1:]:
                conf = conf & c
            confirmations_ok = conf
        else:
            confirm_count = sum(c.astype(int) for c in confirms)
            confirmations_ok = confirm_count >= int(cfg.min_confirmations)

    entry_signal = st_flip_up & confirmations_ok

    st_flip_down = (d["st_dir"].shift(1) == 1) & (d["st_dir"] == -1)
    macd_bear   = (d["macd_line"] < d["macd_signal"]) if cfg.use_macd_confirm else pd.Series(False, index=d.index)
    ema_bear    = (d["ema_f"] < d["ema_s"]) if cfg.use_ema_confirm else pd.Series(False, index=d.index)
    ema921_bear = (d["ema_f2"] < d["ema_s2"]) if cfg.use_ema921_confirm else pd.Series(False, index=d.index)
    rsi_ob      = (d["rsi"] >= cfg.rsi_ob_level) if cfg.use_rsi_filter else pd.Series(False, index=d.index)
    di_bear     = (d["di_minus"] > d["di_plus"]) if cfg.use_adx_confirm else pd.Series(False, index=d.index)

    in_pos = False
    entry_px = stop_px = tgt_px = 0.0
    trades = []

    idx = list(d.index)
    for i in range(len(idx)-1):
        dt, nxt = idx[i], idx[i+1]
        row, nxt_row = d.loc[dt], d.loc[nxt]

        if not in_pos:
            if entry_signal.loc[dt] and basic_liquidity_ok(row, cfg):
                px = float(nxt_row["Open"] if cfg.entry_on_next_open else row["Close"])
                sig_bits = ["ST flip ↑"]
                if cfg.use_rsi_filter:     sig_bits.append(f"RSI<{cfg.rsi_ob_level:g}" + (" & rising" if cfg.rsi_require_rising else ""))
                if cfg.use_macd_confirm:   sig_bits.append("MACD≥Signal or recent bull cross")
                if cfg.use_ema_confirm:    sig_bits.append("EMA5>EMA20")
                if cfg.use_ema921_confirm: sig_bits.append("EMA9>EMA21")
                if cfg.use_bb_reinforce:   sig_bits.append("Close≥UpperBB & mid rising")
                if cfg.use_obv_confirm:    sig_bits.append("OBV rising")
                if cfg.use_adx_confirm:    sig_bits.append(f"ADX≥{cfg.adx_min:g} & +DI≥−DI")
                if cfg.min_confirmations is not None: sig_bits.append(f"min_conf={cfg.min_confirmations}")

                trades.append({
                    "ticker": ticker, "side": "BUY", "date": nxt if cfg.entry_on_next_open else dt,
                    "price": px, "shares": 0,
                    "reason": "candidate",
                    "signal_reason": "; ".join(sig_bits),
                    "score": float(0.0),
                    "rsi": float(row["rsi"]), "macd_line": float(row["macd_line"]), "macd_signal": float(row["macd_signal"]),
                    "ema_f": float(row["ema_f"]), "ema_s": float(row["ema_s"]),
                    "ema_f2": float(row["ema_f2"]), "ema_s2": float(row["ema_s2"]),
                    "st_value": float(row["st_value"]), "st_dir": int(row["st_dir"]),
                    "bb_mid": float(row["bb_mid"]), "bb_up": float(row["bb_up"]),
                    "obv": float(row["obv"]), "adx": float(row["adx"]),
                    "di_plus": float(row["di_plus"]), "di_minus": float(row["di_minus"]),
                    "close": float(row["Close"]), "high_52w": float(row["high_52w"])
                })
                in_pos = True
                entry_px = px
                stop_px = entry_px * (1 - cfg.stop_loss_pct)
                tgt_px  = entry_px * (1 + cfg.target_pct)
        else:
            hit = None
            exec_date = nxt if cfg.exit_on_next_open else dt

            if nxt_row["Low"] <= stop_px and nxt_row["High"] >= tgt_px:
                hit, exec_price = "target", float(tgt_px)
            elif nxt_row["Low"] <= stop_px:
                hit, exec_price = "stop", float(stop_px)
            elif nxt_row["High"] >= tgt_px:
                hit, exec_price = "target", float(tgt_px)
            elif st_flip_down.loc[dt]:
                hit, exec_price = "st_bear_flip", float(nxt_row["Open"] if cfg.exit_on_next_open else row["Close"])
            elif macd_bear.loc[dt]:
                hit, exec_price = "macd_bear", float(nxt_row["Open"] if cfg.exit_on_next_open else row["Close"])
            elif ema_bear.loc[dt]:
                hit, exec_price = "ema5_20_bear", float(nxt_row["Open"] if cfg.exit_on_next_open else row["Close"])
            elif ema921_bear.loc[dt]:
                hit, exec_price = "ema9_21_bear", float(nxt_row["Open"] if cfg.exit_on_next_open else row["Close"])
            elif di_bear.loc[dt]:
                hit, exec_price = "di_bear", float(nxt_row["Open"] if cfg.exit_on_next_open else row["Close"])
            elif rsi_ob.loc[dt]:
                hit, exec_price = "rsi_overbought", float(nxt_row["Open"] if cfg.exit_on_next_open else row["Close"])

            if hit is not None:
                trades.append({
                    "ticker": ticker, "side": "SELL", "date": exec_date,
                    "price": float(exec_price), "shares": 0, "reason": hit,
                    "signal_reason": "",
                    "score": np.nan,
                    "rsi": float(row["rsi"]), "macd_line": float(row["macd_line"]), "macd_signal": float(row["macd_signal"]),
                    "ema_f": float(row["ema_f"]), "ema_s": float(row["ema_s"]),
                    "ema_f2": float(row["ema_f2"]), "ema_s2": float(row["ema_s2"]),
                    "st_value": float(row["st_value"]), "st_dir": int(row["st_dir"]),
                    "bb_mid": float(row["bb_mid"]), "bb_up": float(row["bb_up"]),
                    "obv": float(row["obv"]), "adx": float(row["adx"]),
                    "di_plus": float(row["di_plus"]), "di_minus": float(row["di_minus"]),
                    "close": float(row["Close"]), "high_52w": float(row["high_52w"])
                })
                in_pos = False
                entry_px = stop_px = tgt_px = 0.0

    if in_pos:
        last_dt = d.index[-1]; row = d.loc[last_dt]
        trades.append({
            "ticker": ticker, "side": "SELL", "date": last_dt,
            "price": float(row["Close"]), "shares": 0, "reason": "final_close",
            "signal_reason": "",
            "score": np.nan,
            "rsi": float(row["rsi"]), "macd_line": float(row["macd_line"]), "macd_signal": float(row["macd_signal"]),
            "ema_f": float(row["ema_f"]), "ema_s": float(row["ema_s"]),
            "ema_f2": float(row["ema_f2"]), "ema_s2": float(row["ema_s2"]),
            "st_value": float(row["st_value"]), "st_dir": int(row["st_dir"]),
            "bb_mid": float(row["bb_mid"]), "bb_up": float(row["bb_up"]),
            "obv": float(row["obv"]), "adx": float(row["adx"]),
            "di_plus": float(row["di_plus"]), "di_minus": float(row["di_minus"]),
            "close": float(row["Close"]), "high_52w": float(row["high_52w"])
        })

    return pd.DataFrame(trades, columns=cols), pd.Series(dtype=float)


In [55]:

def pick_benchmark(benchmarks: Tuple[str,...], start: str, end: Optional[str], cache_dir: str) -> Tuple[str, pd.DataFrame]:
    for t in ("^CRSLDX", "^CNX500","^NIFTY500","^NSEI","^BSESN","^BSE500"):
        try:
            data = fetch_prices([t], start, end, cache_dir)
            df = data.get(t)
            if df is not None and not df.empty:
                log.info("Using benchmark: %s", t)
                return t, df
        except Exception:
            pass
    idx = pd.date_range(start=start, end=end or today_str(), freq="B")
    df = pd.DataFrame({"Close": np.ones(len(idx))}, index=idx)
    log.warning("No benchmark found; using synthetic flat series.")
    return "SYNTH_BENCH", df

def compute_volar_scores(end_dt: pd.Timestamp, tickers: List[str], data_map: Dict[str,pd.DataFrame], bench_df: pd.DataFrame, lookback: int) -> Dict[str, float]:
    scores = {}
    bser = bench_df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
    for t in tickers:
        df = data_map.get(t); scores[t] = 0.0
        if df is None or df.empty:
            continue
        if end_dt not in df.index:
            df = df[df.index <= end_dt]
            if df.empty: continue
        r = df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
        common = pd.concat([r, bser], axis=1, keys=["s","b"]).dropna()
        if common.shape[0] < max(20, int(0.4*lookback)):
            continue
        excess = common["s"] - common["b"]
        vol = common["s"].std(ddof=0)
        scores[t] = 0.0 if vol <= 1e-8 else float((excess.mean() / vol) * math.sqrt(252.0))
    return scores


In [56]:

def markowitz_long_only(mu: np.ndarray, Sigma: np.ndarray) -> np.ndarray:
    n = len(mu); eps = 1e-6
    Sigma = Sigma + eps*np.eye(n)
    def solve_lambda(lmbd: float, active_mask=None):
        ones = np.ones(n if active_mask is None else np.count_nonzero(active_mask))
        if active_mask is None:
            A = np.block([[2*lmbd*Sigma, np.ones((n,1))],[np.ones((1,n)), np.zeros((1,1))]])
            b = np.concatenate([mu, np.array([1.0])])
            try:
                sol = np.linalg.solve(A, b); w = sol[:n]
            except np.linalg.LinAlgError:
                w = np.full(n, 1.0/n)
            return w
        else:
            idx = np.where(active_mask)[0]
            if len(idx)==0: return np.full(n, 1.0/n)
            S = Sigma[np.ix_(idx, idx)]; o = np.ones(len(idx)); m = mu[idx]
            A = np.block([[2*lmbd*S, o[:,None]],[o[None,:], np.zeros((1,1))]]); b = np.concatenate([m, np.array([1.0])])
            try:
                sol = np.linalg.solve(A, b); w_sub = sol[:len(idx)]
            except np.linalg.LinAlgError:
                w_sub = np.full(len(idx), 1.0/len(idx))
            w = np.zeros(n); w[idx] = w_sub; return w
    best_w = np.full(n, 1.0/n); best_sr = -1e9
    for lmbd in np.logspace(-3, 3, 31):
        active = np.ones(n, dtype=bool); w = None
        for _ in range(n):
            w = solve_lambda(lmbd, active_mask=active)
            neg = w < 0
            if not neg.any(): break
            worst = np.argmin(w); active[worst] = False
        if w is None: continue
        w = np.clip(w, 0, None); 
        if w.sum() <= 0: continue
        w = w / w.sum()
        mu_p = float(mu @ w); vol_p = float(np.sqrt(w @ Sigma @ w))
        if vol_p <= 1e-8: continue
        sr = mu_p / vol_p
        if sr > best_sr: best_sr, best_w = sr, w.copy()
    return best_w


In [57]:

def aggregate_and_apply(all_trades: pd.DataFrame, data_map: Dict[str, pd.DataFrame], bench_df: pd.DataFrame, cfg: Config):
    if all_trades.empty:
        return all_trades, pd.DataFrame(), pd.Series(dtype=float), {}

    side_order = {"BUY": 0, "SELL": 1}
    all_trades = (
        all_trades.assign(_sorder=all_trades["side"].map(side_order))
        .sort_values(by=["date", "_sorder"], kind="stable")
        .drop(columns=["_sorder"])
        .reset_index(drop=True)
    )
    all_trades["date"] = pd.to_datetime(all_trades["date"])

    equity_curve = []
    dates = sorted(all_trades["date"].unique().tolist())
    cash = cfg.initial_capital
    open_positions = {}
    completed_legs = []

    global APPLY_FEES
    APPLY_FEES = cfg.apply_fees

    def _get_close_on(tkr, dt):
        df = data_map.get(tkr)
        if df is None or df.empty: return np.nan
        if dt in df.index: return float(df.loc[dt, "Close"])
        prev = df[df.index <= dt]
        if prev.empty: return np.nan
        return float(prev["Close"].iloc[-1])

    if dates:
        seed_date = pd.to_datetime(dates[0]) - pd.Timedelta(days=1)
        equity_curve.append((seed_date, float(cash)))

    for dt in dates:
        day_trades = all_trades[all_trades["date"] == dt].copy()

        # SELL first
        for _, tr in day_trades[day_trades["side"] == "SELL"].iterrows():
            tkr = tr["ticker"]; price = float(tr["price"])
            pos = open_positions.get(tkr)
            if pos is None: continue
            shares = int(pos["shares"])
            turnover_sell = shares * price
            fee = calc_fees(0.0, turnover_sell)
            pnl = (price - pos["entry_px"]) * shares
            cash += (turnover_sell - fee)
            realized = pnl - fee - pos.get("buy_fee", 0.0)

            entry_dt = pd.to_datetime(pos["entry_date"]); exit_dt  = pd.to_datetime(dt)
            days_held = int((exit_dt - entry_dt).days)
            exit_reason_text = tr.get("reason", "")

            completed_legs.append({
                "ticker": tkr, "side": "SELL", "date": dt, "price": price, "shares": shares,
                "reason": exit_reason_text,
                "entry_reason": pos.get("entry_reason", ""),
                "entry_signal_reason": pos.get("entry_signal_reason", ""),
                "exit_reason": exit_reason_text,
                "days_held": days_held,
                "turnover": turnover_sell, "fees_inr": fee, "pnl_inr": realized,
            })
            del open_positions[tkr]

        # BUY candidates
        buys_today = day_trades[day_trades["side"] == "BUY"].copy()

        # 52w filter
        if not buys_today.empty:
            keep = []
            for _, rr in buys_today.iterrows():
                df = data_map.get(rr["ticker"])
                if df is None or df.empty or dt not in df.index: continue
                close = float(df.loc[dt, "Close"])
                hist = df["Close"].loc[:dt]
                window = hist.iloc[-cfg.filter_52w_window:] if len(hist)>=cfg.filter_52w_window else hist
                high_52w = float(window.max())
                if high_52w>0 and close >= cfg.within_pct_of_52w_high * high_52w:
                    keep.append(rr)
            buys_today = pd.DataFrame(keep) if keep else pd.DataFrame(columns=buys_today.columns)

        if not buys_today.empty:
            buys_today = buys_today[~buys_today["ticker"].isin(open_positions.keys())]

        if not buys_today.empty:
            bench_tkr, bench_df2 = pick_benchmark((), CFG.start_date, CFG.end_date, CFG.cache_dir)
            tickers = buys_today["ticker"].tolist()
            volar_scores = compute_volar_scores(dt, tickers, data_map, bench_df2, cfg.volar_lookback)
            buys_today["volar"] = buys_today["ticker"].map(volar_scores)
            buys_today = buys_today.sort_values("volar", ascending=False).reset_index(drop=True)

        slots = cfg.max_concurrent_positions - len(open_positions)
        selected = pd.DataFrame(columns=buys_today.columns)
        if slots > 0 and not buys_today.empty:
            selected = buys_today.head(min(cfg.top_k_daily, slots)).copy()

        if not selected.empty:
            names = selected["ticker"].tolist()
            rets = []
            for t in names:
                df = data_map.get(t)
                ser = df["Close"].loc[:dt].pct_change().dropna().iloc[-cfg.volar_lookback:]
                rets.append(ser)
            R = pd.concat(rets, axis=1); R.columns = names; R = R.dropna()
            if R.empty or R.shape[0] < max(20, int(0.4*cfg.volar_lookback)) or R.shape[1] == 0:
                weights = np.full(len(names), 1.0/len(names))
            else:
                mu = R.mean().values; Sigma = R.cov().values; weights = markowitz_long_only(mu, Sigma)

            deploy_cash = max(0.0, float(cash)) * float(cfg.deploy_cash_frac)
            if deploy_cash > 0:
                alloc = (weights / weights.sum()) * deploy_cash if weights.sum()>0 else np.full(len(names), deploy_cash/len(names))
                for w_amt, t in zip(alloc, names):
                    df_t = data_map[t]
                    price = float(df_t.loc[dt, "Close"] if dt in df_t.index else df_t["Close"].loc[:dt].iloc[-1])
                    shares = int(math.floor(w_amt / price))
                    if shares <= 0: continue
                    turn = shares * price
                    fee = calc_fees(turn, 0.0)
                    total_cost = turn + fee
                    if total_cost > cash:
                        shares = int(math.floor((cash - fee) / price))
                        if shares <= 0: continue
                        turn = shares * price; total_cost = turn + fee
                    cash -= total_cost

                    row_sel = selected[selected["ticker"] == t].iloc[0]
                    entry_reason_text = "SuperTrend entry"
                    entry_signal_reason = str(row_sel.get("signal_reason", ""))

                    open_positions[t] = {"entry_date": dt, "entry_px": price, "shares": shares,
                                         "buy_fee": fee, "entry_reason": entry_reason_text,
                                         "entry_signal_reason": entry_signal_reason}

                    completed_legs.append({
                        "ticker": t, "side": "BUY", "date": dt, "price": price, "shares": shares,
                        "reason": entry_reason_text,
                        "entry_reason": entry_reason_text,
                        "entry_signal_reason": entry_signal_reason,
                        "exit_reason": "",
                        "days_held": 0,
                        "turnover": turn, "fees_inr": fee, "pnl_inr": 0.0,
                    })

        mtm = 0.0
        for _tkr, pos in open_positions.items():
            px = _get_close_on(_tkr, dt)
            if not np.isnan(px): mtm += pos["shares"] * px
        total_equity = cash + mtm
        equity_curve.append((dt, float(total_equity)))

    eq_ser = pd.Series([e for _, e in equity_curve], index=[d for d, _ in equity_curve])
    legs_df = pd.DataFrame(completed_legs).sort_values(["date", "ticker", "side"]).reset_index(drop=True)

    trips = []
    by_tkr_open = {}
    for _, leg in legs_df.iterrows():
        t = leg["ticker"]
        if leg["side"] == "BUY":
            by_tkr_open[t] = leg
        else:
            buy = by_tkr_open.pop(t, None)
            if buy is None: continue
            fees_total = float(buy.get("fees_inr", 0.0) + leg.get("fees_inr", 0.0))
            gross_pnl = (leg["price"] - buy["price"]) * buy["shares"]
            net_pnl   = gross_pnl - fees_total
            entry_date = pd.to_datetime(buy["date"])
            exit_date  = pd.to_datetime(leg["date"])
            days_held  = int((exit_date - entry_date).days)
            trips.append({
                "ticker": t,
                "entry_date": entry_date,
                "entry_price": float(buy["price"]),
                "exit_date": exit_date,
                "exit_price": float(leg["price"]),
                "days_held": days_held,
                "shares": int(buy["shares"]),
                "entry_reason": buy.get("entry_reason", buy.get("reason", "")),
                "entry_signal_reason": buy.get("entry_signal_reason", ""),
                "exit_reason": leg.get("exit_reason", leg.get("reason", "")),
                "net_pnl_inr": float(net_pnl),
                "return_pct": float((leg["price"]/buy["price"]-1.0)*100.0)
            })
    trips_df = pd.DataFrame(trips).sort_values(["entry_date","ticker"]).reset_index(drop=True)

    metrics = compute_metrics(eq_ser, legs_df)
    return legs_df, trips_df, eq_ser, metrics


In [58]:

def compute_metrics(equity: pd.Series, legs_df: pd.DataFrame):
    out = {}
    if equity is None or equity.empty: return out
    eq = equity.dropna()
    daily_ret = eq.pct_change().fillna(0.0)

    days = (eq.index[-1] - eq.index[0]).days or 1
    years = days / 365.25
    cagr = (eq.iloc[-1] / eq.iloc[0]) ** (1/years) - 1 if years > 0 else 0.0

    sharpe = (daily_ret.mean() / daily_ret.std(ddof=0) * np.sqrt(252)) if daily_ret.std(ddof=0) > 0 else 0.0

    cummax = eq.cummax()
    dd = (eq - cummax) / cummax
    max_dd = dd.min()

    wins = 0
    n_sells = legs_df[legs_df["side"] == "SELL"].shape[0] if legs_df is not None and not legs_df.empty else 0
    for _, r in legs_df[legs_df["side"] == "SELL"].iterrows():
        if float(r.get("pnl_inr", 0.0)) > 0:
            wins += 1
    win_rate = (wins / n_sells) * 100.0 if n_sells > 0 else 0.0

    out.update({
        "start_equity_inr": float(eq.iloc[0]),
        "final_equity_inr": float(eq.iloc[-1]),
        "cagr_pct": float(cagr * 100),
        "sharpe": float(sharpe),
        "max_drawdown_pct": float(max_dd * 100),
        "win_rate_pct": float(win_rate),
        "n_trades": int(n_sells),
    })
    return out


In [59]:

def plot_equity(equity: pd.Series, out_path: str):
    if equity is None or equity.empty:
        return
    try:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10,5))
        plt.plot(equity.index, equity.values)
        plt.title("Equity Curve")
        plt.xlabel("Date"); plt.ylabel("Equity (INR)")
        plt.tight_layout(); plt.savefig(out_path); plt.close()
    except Exception:
        pass

def backtest(cfg: Config):
    ensure_dirs(cfg.cache_dir, cfg.out_dir)
    log.info("Universe: loading static symbols...")
    symbols = load_static_symbols(cfg.static_symbols, cfg.static_symbols_path)
    log.info("Loaded %d symbols.", len(symbols))

    log.info("Data: fetching OHLCV from yfinance (adjusted)...")
    data_map = fetch_prices(symbols, cfg.start_date, cfg.end_date, cfg.cache_dir)
    log.info("Downloaded %d symbols with data.", len(data_map))

    bench_tkr, bench_df = pick_benchmark((), cfg.start_date, cfg.end_date, cfg.cache_dir)
    log.info("Benchmark selected: %s", bench_tkr)

    log.info("Signals: generating SuperTrend entries with optional confirmations...")
    all_trades = []
    for i, tkr in enumerate(symbols, 1):
        df = data_map.get(tkr)
        if df is None or df.empty: continue
        tr, _ = simulate_ticker(tkr, df, cfg)
        if not tr.empty: all_trades.append(tr)
        if i % 50 == 0: log.info("  processed %d/%d tickers...", i, len(symbols))

    if not all_trades:
        log.warning("No signals generated; consider relaxing toggles/thresholds or lowering min_confirmations.")
        return None, None, None, {}
    all_trades = pd.concat(all_trades, ignore_index=True)

    log.info("Portfolio: cap daily deploy to %.0f%% of cash; 52w>=%.0f%% high; top-%d by VOLAᵣ; MVO; max %d positions.",
             cfg.deploy_cash_frac*100, cfg.within_pct_of_52w_high*100, cfg.top_k_daily, cfg.max_concurrent_positions)
    legs_df, trips_df, equity, metrics = aggregate_and_apply(all_trades, data_map, bench_df, cfg)

    stamp = pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y%m%d_%H%M%S")
    ensure_dirs(cfg.out_dir)
    legs_path = os.path.join(cfg.out_dir, f"trades_legs_{stamp}.csv")
    trips_path = os.path.join(cfg.out_dir, f"trades_roundtrips_{stamp}.csv")
    equity_path = os.path.join(cfg.out_dir, f"equity_{stamp}.csv")
    metrics_path = os.path.join(cfg.out_dir, f"metrics_{stamp}.json")
    eq_plot_path = os.path.join(cfg.out_dir, f"equity_{stamp}.png")

    if legs_df is not None: legs_df.to_csv(legs_path, index=False)
    if trips_df is not None: trips_df.to_csv(trips_path, index=False)
    if equity is not None: pd.DataFrame({"date": equity.index, "equity": equity.values}).to_csv(equity_path, index=False)
    with open(metrics_path, "w") as f: json.dump(metrics, f, indent=2)
    if cfg.plot and equity is not None: plot_equity(equity, eq_plot_path)

    log.info("=== METRICS ===\n%s", json.dumps(metrics, indent=2))
    log.info("Files written:\n  %s\n  %s\n  %s\n  %s", legs_path, trips_path, equity_path, metrics_path)
    if cfg.plot: log.info("  %s", eq_plot_path)
    return legs_df, trips_df, equity, metrics


In [60]:

# Example: tiny smoke test
CFG.static_symbols = ['360ONE.NS', '3MINDIA.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABLBL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ACMESOLAR.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AEGISLOG.NS', 'AEGISVOPAK.NS', 'AFCONS.NS', 'AFFLE.NS', 'AGARWALEYE.NS', 'AIAENG.NS', 'AIIL.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'AKZOINDIA.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ARE&M.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATHERENERG.NS', 'ATUL.NS', 'AUBANK.NS', 'AUROPHARMA.NS', 'AWL.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJAJHFL.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBTC.NS', 'BDL.NS', 'BEL.NS', 'BEML.NS', 'BERGEPAINT.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BHEL.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUEJET.NS', 'BLUESTARCO.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMPUS.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOICEIN.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIPLA.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CONCOR.NS', 'CONCORDBIO.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DBREALTY.NS', 'DCMSHRIRAM.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DMART.NS', 'DOMS.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ENRIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'FACT.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FIRSTCRY.NS', 'FIVESTAR.NS', 'FLUOROCHEM.NS', 'FORCEMOT.NS', 'FORTIS.NS', 'FSL.NS', 'GAIL.NS', 'GESHIP.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GMDCLTD.NS', 'GMRAIRPORT.NS', 'GODFRYPHLP.NS', 'GODIGIT.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GPIL.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GRSE.NS', 'GSPL.NS', 'GUJGASLTD.NS', 'GVT&D.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEROMOTOCO.NS', 'HEXT.NS', 'HFCL.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HSCL.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFCI.NS', 'IGIL.NS', 'IGL.NS', 'IIFL.NS', 'IKS.NS', 'INDGN.NS', 'INDHOTEL.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIGO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXINDIA.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'ITCHOTELS.NS', 'ITI.NS', 'J&KBANK.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JKCEMENT.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KAJARIACER.NS', 'KALYANKJIL.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KEI.NS', 'KFINTECH.NS', 'KIMS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KPIL.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'KSB.NS', 'LALPATHLAB.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LT.NS', 'LTF.NS', 'LTFOODS.NS', 'LTIM.NS', 'LTTS.NS', 'LUPIN.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHSCOOTER.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANKIND.NS', 'MANYAVAR.NS', 'MAPMYINDIA.NS', 'MARICO.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'MEDANTA.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOTHERSON.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MRF.NS', 'MRPL.NS', 'MSUMI.NS', 'MUTHOOTFIN.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NBCC.NS', 'NCC.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIVABUPA.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPC.NS', 'NTPCGREEN.NS', 'NUVAMA.NS', 'NUVOCO.NS', 'NYKAA.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'ONESOURCE.NS', 'ONGC.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PAYTM.NS', 'PCBL.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PGHH.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PNB.NS', 'PNBHOUSING.NS', 'POLICYBZR.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POONAWALLA.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'PPLPHARMA.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PTCIL.NS', 'PVRINOX.NS', 'RADICO.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RAMCOCEM.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RELINFRA.NS', 'RHIM.NS', 'RITES.NS', 'RKFORGE.NS', 'RPOWER.NS', 'RRKABEL.NS', 'RVNL.NS', 'SAGILITY.NS', 'SAIL.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'SRF.NS', 'STARHEALTH.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TARIL.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TBOTEK.NS', 'TCS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'THELEELA.NS', 'THERMAX.NS', 'TIINDIA.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UNITDSPR.NS', 'UNOMINDA.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'VBL.NS', 'VEDL.NS', 'VENTIVE.NS', 'VGUARD.NS', 'VIJAYA.NS', 'VMM.NS', 'VOLTAS.NS', 'VTL.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ZFCVINDIA.NS', 'ZYDUSLIFE.NS']

CFG.plot = True

CFG.min_confirmations = 1
CFG.use_macd_confirm = False
CFG.use_ema_confirm = True
CFG.use_ema921_confirm = False
CFG.use_adx_confirm = False

CFG.use_rsi_filter = False
CFG.use_bb_reinforce = True
CFG.use_obv_confirm = False

legs_df, trips_df, equity, metrics = backtest(CFG)
metrics


2025-10-12 12:05:21 | INFO | Universe: loading static symbols...
2025-10-12 12:05:21 | INFO | Loaded 500 symbols.
2025-10-12 12:05:21 | INFO | Data: fetching OHLCV from yfinance (adjusted)...
2025-10-12 12:05:25 | ERROR | 
1 Failed download:
2025-10-12 12:05:25 | ERROR | ['ABLBL.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420050600, endDate = 1735669800")')
2025-10-12 12:05:29 | ERROR | 
1 Failed download:
2025-10-12 12:05:29 | ERROR | ['AEGISVOPAK.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420050600, endDate = 1735669800")')
2025-10-12 12:05:30 | ERROR | 
1 Failed download:
2025-10-12 12:05:30 | ERROR | ['AGARWALEYE.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420050600

{'start_equity_inr': 500000.0,
 'final_equity_inr': 552964.8454066671,
 'cagr_pct': 1.1334263969628777,
 'sharpe': 0.21050133193237588,
 'max_drawdown_pct': -37.022719232346475,
 'win_rate_pct': 37.44444444444445,
 'n_trades': 900}